In [41]:
from collections import deque
import math

In [8]:
class Module:
    def __init__(self, name, type, outputs):
        self.name = name
        self.type = type
        self.outputs = outputs

        if type == "%":
            self.memory = "off"
        else:
            self.memory = {}
    
    def __repr__(self):
        return f"name: {self.name}, type: {self.type}, outputs: {', '.join(self.outputs)}, memory: {str(self.memory)}"

In [34]:
def read_input(filename):
    modules = {}
    with open(filename) as f:
        for line in f:
            left, right = line.strip().split(" -> ")
            outputs = right.split(", ")
            if left == "broadcaster":
                broadcast_targets = outputs
            else:
                type = left[0]
                name = left[1:]
                modules[name] = Module(name, type, outputs)

    for name, module in modules.items():
        for output in module.outputs:
            if output in modules and modules[output].type == "&":
                modules[output].memory[name] = "lo"

    return modules, broadcast_targets

In [35]:
modules, broadcast_targets = read_input("20_input.txt")
lo = hi = 0
for _ in range(1000):
    lo += 1
    q = deque([("broadcaster", x, "lo") for x in broadcast_targets])
    while q:
        origin, target, pulse = q.popleft()
        if pulse == "lo":
            lo += 1
        else:
            hi += 1
        if target not in modules:
            continue
        module = modules[target]
        if module.type == "%":
            if pulse == "lo":
                module.memory = "on" if module.memory == "off" else "off"
                outgoing = "hi" if module.memory == "on" else "lo"
                for x in module.outputs:
                    q.append((module.name, x, outgoing))
        else:
            module.memory[origin] = pulse
            outgoing = "lo" if all(x == "hi" for x in module.memory.values()) else "hi"
            for x in module.outputs:
                q.append((module.name, x, outgoing))

In [36]:
lo * hi

825896364

In [49]:
modules, broadcast_targets = read_input("20_input.txt")
(feed,) = [name for name, module in modules.items() if "rx" in module.outputs]
cycle_lengths = {}
seen = {name: 0 for name, module in modules.items() if feed in module.outputs}

In [50]:
presses = 0
enough = False
while True:
    presses += 1
    q = deque([("broadcaster", x, "lo") for x in broadcast_targets])
    while q:
        origin, target, pulse = q.popleft()
        if target not in modules:
            continue
        module = modules[target]

        if module.name == feed and pulse == "hi":
            seen[origin] += 1
            if origin not in cycle_lengths:
                cycle_lengths[origin] = presses
            else:
                assert presses == seen[origin] * cycle_lengths[origin]

            if all(x > 10 for x in seen.values()):
                enough = True
                break

        if module.type == "%":
            if pulse == "lo":
                module.memory = "on" if module.memory == "off" else "off"
                outgoing = "hi" if module.memory == "on" else "lo"
                for x in module.outputs:
                    q.append((module.name, x, outgoing))
        else:
            module.memory[origin] = pulse
            outgoing = "lo" if all(x == "hi" for x in module.memory.values()) else "hi"
            for x in module.outputs:
                q.append((module.name, x, outgoing))
    if enough:
        break

In [54]:
math.lcm(*cycle_lengths.values())

243566897206981